In [1]:
# Initialise 
!pip install --upgrade --force-reinstall boto3 
!pip install openpyxl

import boto3
import sys
import pandas as pd
from botocore.exceptions import ClientError

print('Running boto3 version:', boto3.__version__)

def load_excel(file_path, sheet_name=0):
    """Load data from an Excel file into a Pandas DataFrame."""
    return pd.read_excel(file_path, sheet_name=sheet_name)

excel_data = load_excel('LOREAL_Data.xlsx')
print(type(excel_data))

  Using cached boto3-1.36.24-py3-none-any.whl.metadata (6.7 kB)
  Using cached botocore-1.36.24-py3-none-any.whl.metadata (5.7 kB)
  Using cached jmespath-1.0.1-py3-none-any.whl.metadata (7.6 kB)
  Using cached s3transfer-0.11.2-py3-none-any.whl.metadata (1.7 kB)
  Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl.metadata (8.4 kB)
  Using cached urllib3-2.3.0-py3-none-any.whl.metadata (6.5 kB)
  Using cached six-1.17.0-py2.py3-none-any.whl.metadata (1.7 kB)
Using cached boto3-1.36.24-py3-none-any.whl (139 kB)
Using cached botocore-1.36.24-py3-none-any.whl (13.4 MB)
Using cached jmespath-1.0.1-py3-none-any.whl (20 kB)
Using cached s3transfer-0.11.2-py3-none-any.whl (84 kB)
Using cached python_dateutil-2.9.0.post0-py2.py3-none-any.whl (229 kB)
Using cached urllib3-2.3.0-py3-none-any.whl (128 kB)
Using cached six-1.17.0-py2.py3-none-any.whl (11 kB)
  Attempting uninstall: urllib3
    Found existing installation: urllib3 2.3.0
    Uninstalling urllib3-2.3.0:
      Successfully

In [2]:
# Define Region/Model
region = 'us-west-2'
print('Using region: ', region)

bedrock = boto3.client(
    service_name = 'bedrock-runtime',
    region_name = region,
    )

model_id = "us.meta.llama3-3-70b-instruct-v1:0"

Using region:  us-west-2


In [3]:
# Define functions
## Model caller
def invoke_bedrock_model(client, id, prompt, description, max_tokens=2000, temperature=0.1, top_p=0.2):
    response = ""
    try:
        response = client.converse(
            modelId=id,
            messages=[
                {
                    "role": "user",
                    "content": [
                        {
                            "text": prompt
                        }
                    ]
                }
            ],
            inferenceConfig={
                "temperature": temperature,
                "maxTokens": max_tokens,
                "topP": top_p
            }
            # additionalModelRequestFields={
            
            # }
        )
    except Exception as e:
        print(e)
        result = "Model invocation error"
    try:
        result = (response['output']['message']['content'][0]['text']) #\
#        + '\n--- Latency: ' + str(response['metrics']['latencyMs']) \
#        + 'ms - Input tokens:' + str(response['usage']['inputTokens']) \
#        + ' - Output tokens:' + str(response['usage']['outputTokens']) + ' ---\n')
        return result
    except Exception as e:
        print(e)
        result = "Output parsing error"
    return result


## Call model for description using prompt
def run_prompt_for_desc(description):
        prompt = (f"""
    ALWAYS use JSON format. You are an expert L'Oreal products with a phd specialised in extracting specific atributes from descriptions. You have been providing the atributes based on a predefined list for a given description for 20 years. Your task is now to read, understand, analyse the following description and extract the atributes requested in the list. You must include ALL the atributes and ALWAYS use the provided options as answers. You Must ALWAYS include the Chain of Thought before providing atributes and at the end you MUST reflect again on your anwser and provide corrections. 
    
    Main keys of the JSON:
    0_CoT
    1_Skin_Concerns
    2_Demographics_Skin_Type
    3_Product_Function_Routine
    4_Time_of_use
    
    The anwser should follow the structure Key:atribute:presence. Here is an exemple: 
    
    Description: 
    ($135 Value) iS Clinical Active Serum, 1 Oz Our most popular   product, this fast-acting iS Clinical Active Serum, long-term,   results-oriented formula decreases the appearance of fine lines and wrinkles,   evens skin tone, and is excellent for acne. Touted by physicians as   remarkable and phenomenal, ACTIVE SERUM usually produces results within a   couple of days. Potent activity will be noted upon application as evidenced   by a cool tingling sensation indicating penetration. Excellent for all skin   types and for all ages, this powerful botanical serum does not dry the skin.   Instead, it leaves the skin moist and smooth. Reduces appearance of fine lines and wrinkles Smooths and softens skin Helps acne-prone skin
    
    Response: 
    0. Chain of Though
    For Skin Concerns:
    Identifying Key Phrases:
    The description mentions “decreases the appearance of fine lines and wrinkles,” which indicates issues related to aging.
    “Evens skin tone” suggests an improvement in overall skin homogeneity.
    “Excellent for acne” and “helps acne-prone skin” clearly point to acne being addressed.
    Mapping to Attributes:
    Since there’s no mention of dark pigmentation, eye contour concerns, issues with pores, or under-eye issues, those are marked as “No.”
    Although “fine lines” are mentioned, the provided interpretation groups them under a combined “wrinkles_fine-lines” attribute, so “fine_lines” is marked “No” while “wrinkles_fine-lines” is marked “Yes.”
    “Lack radiance” is interpreted as being improved by the product (i.e., the product works on dullness), so it is marked “Yes.”
    For Demographics & Skin Type:
    Age & Skin Types:
    The phrase “excellent for all ages” supports marking 18–34, 35–54, and 55–99 as “Yes.”
    “Excellent for all skin types” covers dry, normal, oily, and combination.
    Sensitivity & Gender:
    “Does not dry the skin” and the absence of any warning suggest no sensitivity issues (thus “no_sensitivity” is Yes, while sensitivity levels are No).
    Although “all ages” might imply unisex use, the interpretation here follows the sample outcome by marking “female” as Yes and “male” as No.
    For Product Function & Routine:
    
    Purpose of the Product:
    Being an “active serum” designed to reduce fine lines, wrinkles, and help with acne indicates its role is treatment.
    There’s no mention of cleansing, prepping, moisturizing, or protecting functions; hence, only “treat” is marked “Yes.”
    For Time of Use:
    
    Usage Indications:
    Even though the description doesn’t state a specific time, active serums are commonly used both in the morning and at night for optimal benefits.
    Thus, both “day” and “night” are marked “Yes.”
    
    
    1_Skin Concerns
    dark_pigmentation: Yes
    acne: Yes
    eye_contour: No
    homogeneity: Yes
    lack_firmness: No
    lack_radiance: Yes
    pores: No
    fine_lines: No
    wrinkles_fine-lines: Yes
    eye-wrinkles: No
    undereye-bags: No
    2_Demographics & Skin Type
    generic: No
    18-34: Yes
    35-54: Yes
    55-99: Yes
    dry: Yes
    normal: Yes
    oily: Yes
    combination: Yes
    sensitivity-high: No
    sensitivity-low: No
    no_sensitivity: Yes
    male: No
    female: Yes
    3_Product Function & Routine
    cleanse: No
    prepare: No
    treat: Yes
    targeted: No
    care: No
    moisturize: No
    protect: No
    4_Time of use
    day: Yes
    night: Yes
    
    Here is the current description you have to extract atributes from: "{description}".
    ALWAYS use JSON format.
    """)
        #print(prompt)
        response = invoke_bedrock_model(bedrock, model_id, prompt, description)
        return response


# For Loop that returns a list of responses from the LLM
def loop_for_data_and_range(source, num_rows=1, start_point=0):
    responses = []
    step = 0
    for row in range(num_rows):
        description = source[row + start_point]
        response = run_prompt_for_desc(description)
        responses.append(response)
        print(step)
        step = step + 1
    return responses

def extract_df(response):  
    attributes = [
    "dark_pigmentation",
    "acne",
    "eye_contour",
    "homogeneity",
    "lack_firmness",
    "lack_radiance",
    "pores",
    "fine_lines",
    "wrinkles_fine-lines",
    "eye-wrinkles",
    "undereye-bags",
    "generic",
    "18-34",
    "35-54",
    "55-99",
    "dry",
    "normal",
    "oily",
    "combination",
    "sensitivity-high",
    "sensitivity-low",
    "no_sensitivity",
    "male",
    "female",
    "cleanse",
    "prepare",
    "treat",
    "targeted",
    "care",
    "moisturize",
    "protect",
    "day",
    "night"
    ]
    # Create the data dictionary starting with the description
    data = {}
    
    # Loop through attributes and set the value to 1 if Yes, 0 if No, or None if not found.
    for attribute in attributes:
        if f'"{attribute}": "Yes"' in response:
            data[attribute] = 1
        elif f'"{attribute}": "No"' in response:
            data[attribute] = 0
        else:
            data[attribute] = None
    
    # Specify the column order so that 'description' comes first
    columns = attributes
    
    # Create the DataFrame with a single row
    df = pd.DataFrame([data], columns=columns)
    
    return df

def loop_extract(responses):
    dfs = []
    for response in responses: 
        extracted_resp = extract_df(response)
        dfs.append(extracted_resp)
    final_df = pd.concat(dfs, ignore_index=True)
    return final_df

In [4]:
attributes = [
    "dark_pigmentation",
    "acne",
    "eye_contour",
    "homogeneity",
    "lack_firmness",
    "lack_radiance",
    "pores",
    "fine_lines",
    "wrinkles_fine-lines",
    "eye-wrinkles",
    "undereye-bags",
    "generic",
    "18-34",
    "35-54",
    "55-99",
    "dry",
    "normal",
    "oily",
    "combination",
    "sensitivity-high",
    "sensitivity-low",
    "no_sensitivity",
    "male",
    "female",
    "cleanse",
    "prepare",
    "treat",
    "targeted",
    "care",
    "moisturize",
    "protect",
    "day",
    "night"
    ]
num_rows=750
# First row is 0
start_point=0

responses = loop_for_data_and_range(source=excel_data['text_raw'], num_rows=num_rows, start_point=start_point)
columns = attributes
final_df = pd.DataFrame(columns=columns)
final_df = loop_extract(responses)
# Here, we're assuming that excel_data['text_raw'] is a pandas Series.
descriptions = excel_data['text_raw'].iloc[start_point : start_point + num_rows]

# Insert the description column as the first column in the final DataFrame
final_df.insert(0, 'description', descriptions.values)

# Create a filename using f-string formatting
file_name = f"final_excel_from_{start_point}_to_{start_point + num_rows}(1).xlsx"

# Save the DataFrame to an Excel file in the current directory
final_df.to_excel(file_name, index=False)

print(f"Saved file: {file_name}")

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [ ]:
import os
import glob
import pandas as pd
import numpy as np

def majority_vote(paths=None, folder_path=None, output_file="final_majority_vote.xlsx"):
    """
    Performs majority voting on multiple Excel files containing multi-label classification results.
    Assumes the first column contains non-numeric descriptions that are kept unchanged.
    
    Parameters:
    - paths: List of file paths to Excel files.
    - folder_path: Path to a folder containing Excel files.
    - output_file: Filename for the final majority voted results.
    
    Returns:
    - Saves the final Excel file with majority voted results.
    """
    file_list = []
    
    # Use provided file paths
    if paths is not None:
        file_list.extend(paths)
    
    # If folder_path is provided, add all .xlsx files from that folder
    if folder_path is not None:
        folder_files = glob.glob(os.path.join(folder_path, "*.xlsx"))
        file_list.extend(folder_files)
    
    # Remove duplicates in case both options were used and overlap
    file_list = list(set(file_list))
    
    if not file_list:
        raise ValueError("No Excel files found. Provide a valid list of paths or folder path.")
    
    # List to store numeric parts of DataFrames (columns 2 to end)
    dfs_numeric = []
    descriptions = None  # To store the descriptions from the first column
    
    for file in file_list:
        try:
            df = pd.read_excel(file)
        except Exception as e:
            print(f"Error reading {file}: {e}")
            continue
        
        # Separate the first column (descriptions)
        if descriptions is None:
            # Keep the first column as is. It will be attached later.
            descriptions = df.iloc[:, [0]]
        # If descriptions already exists, you might want to check consistency here.
        
        # Process numeric columns: columns 2 to end
        df_numeric = df.iloc[:, 1:].apply(pd.to_numeric, errors='coerce')
        # Replace NaN (empty cells or non-numeric cells) with 0.5 (neutral value)
        df_numeric = df_numeric.fillna(0.5)
        dfs_numeric.append(df_numeric)
    
    if not dfs_numeric:
        raise ValueError("No valid DataFrames loaded from the provided files.")
    
    # Stack all numeric parts into a 3D NumPy array. Assumes all DataFrames have the same shape.
    arr = np.array([df.values for df in dfs_numeric], dtype=float)
    
    # Compute the average for each cell across all runs
    avg = np.mean(arr, axis=0)
    
    # Majority vote: if average > 0.5 then 1, else 0.
    majority_vote_arr = (avg > 0.5).astype(int)
    
    # Create a DataFrame for the numeric results using the columns and index from the first numeric DataFrame
    final_numeric_df = pd.DataFrame(majority_vote_arr, columns=dfs_numeric[0].columns, index=dfs_numeric[0].index)
    
    # Combine the descriptions (first column) with the majority voted numeric results
    final_df = pd.concat([descriptions, final_numeric_df], axis=1)
    
    # Save the final DataFrame to a new Excel file
    final_df.to_excel(output_file, index=False)
    
    print(f"Final majority voted Excel file saved as '{output_file}'.")

path = "/Users/sebastian/Documents/Jup for run/Excels L'Oreal/final_excel_from_0_to_750"
majority_vote(folder_path=path)

final_df